# PotPlantSpotter
### Author : Laurin Schweers

Scan individual self created QR code from your plant to access a file with informations and Pouring/Fertilizer Schedule.
You can keep track of of everything by manually input the data into the file.

Later on run it on a local webserver.

## ReadMe:

- First Cell : Installations.
- Second Cell: Only to create the QR codes.
-  

In [4]:
!pip install qrcode[pil] pyzbar opencv-python pandas
!pip install qrcode fpdf
!pip install flask pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [24]:
import qrcode
from fpdf import FPDF

def generate_qr(plant_id):
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    
    qr.add_data(plant_id)
    qr.make(fit=True)

    img = qr.make_image(fill='black', back_color='white')
    img_filename = f'{plant_id}.png'
    img.save(img_filename)
    return img_filename

def create_pdf_with_qr(plant_ids, pdf_filename='qr_codes.pdf'):
    '''
    Takes your qr codes generated with the "generate_qr" method
    and returns all on a DinA4 .pdf for just if u want to print them 
    out all together.

    Input  -> def generate_qr(plant_id)

    Output -> .pdf DinA4 file with all Input QR codes
    
    '''
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()

    x, y = 10, 10
    for plant_id in plant_ids:
        qr_image = generate_qr(plant_id)
        pdf.image(qr_image, x=x, y=y, w=50)
        x += 60
        if x > 160:
            x = 10
            y += 60
        if y > 250:
            pdf.add_page()
            x, y = 10, 10

    pdf.output(pdf_filename)
    print(f'PDF created: {pdf_filename}')

input = ["plant1","plant2"]
type(input)

list

In [1]:
import pandas as pd
from datetime import datetime

def update_plant_data():
    '''
    The function will take a command to update your .csv File with values you assign.
    It will assign the local date to your entry.
    '''


    
    filename = 'plant_data.csv'
    
    # Get the date, plant ID, water amount, and fertilizer amount from the user
    date_today = input("Enter the date (YYYY-MM-DD) or press Enter to use today's date: ")
    if not date_today:
        date_today = datetime.now().strftime('%Y-%m-%d')
    
    plant_id = input("Enter the Plant ID: ")
    water_amount = input("Enter the Water Amount: ")
    fertilizer_amount = input("Enter the Fertilizer Amount: ")

    try:
        # Load existing data
        df = pd.read_csv(filename)
    except FileNotFoundError:
        # Create a new dataframe if the file does not exist
        df = pd.DataFrame(columns=['Date', 'Plant_ID', 'Water_Amount', 'Fertilizer_Amount'])

    # Check if there's already an entry for the specified date and plant
    mask = (df['Date'] == date_today) & (df['Plant_ID'] == plant_id)
    if not df[mask].empty:
        df.loc[mask, 'Water_Amount'] = water_amount
        df.loc[mask, 'Fertilizer_Amount'] = fertilizer_amount
    else:
        # Append new data
        new_row = {'Date': date_today, 'Plant_ID': plant_id, 'Water_Amount': water_amount, 'Fertilizer_Amount': fertilizer_amount}
        df = df.append(new_row, ignore_index=True)

    # Save data back to CSV
    df.to_csv(filename, index=False)

# Example usage
update_plant_data()

In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
from datetime import datetime



'''
LOCAL WEBSERVER

Flask(__name__): Creates a Flask web application.
app.route:       Defines the routes for getting and updating data.

load_data(): Reads the CSV file into a DataFrame.

save_data(df): Saves the DataFrame back to the CSV file.

/get_data: A GET endpoint to fetch data for a specific plant ID.
    Retrieves the plant_id from the query parameters.
    Filters the DataFrame for the specific plant ID and returns the data as JSON

/update_data: A POST endpoint to update the data.

    Expects JSON data with plant_id, water_amount, and fertilizer_amount.
    Checks if there's already an entry for today and the plant ID.
    Updates the existing entry or appends a new one.
    Saves the updated DataFrame.    
'''

app = Flask(__name__)
filename = 'plant_data.csv'

@app.route('/')
def index():
    return '''
        <form action="/update_data" method="post">
            <input type="text" name="plant_id" placeholder="Plant ID"><br>
            <input type="text" name="water_amount" placeholder="Water Amount"><br>
            <input type="text" name="fertilizer_amount" placeholder="Fertilizer Amount"><br>
            <input type="submit" value="Update">
        </form>
    '''

def load_data():
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return pd.DataFrame(columns=['Date', 'Plant_ID', 'Water_Amount', 'Fertilizer_Amount'])

def save_data(df):
    df.to_csv(filename, index=False)

@app.route('/get_data', methods=['GET'])
def get_data():
    plant_id = request.args.get('plant_id')
    df = load_data()
    data = df[df['Plant_ID'] == plant_id].to_dict(orient='records')
    return jsonify(data)

@app.route('/update_data', methods=['POST'])
def update_data():
    if request.is_json:
        data = request.json
    else:
        data = request.form
    
    plant_id = data['plant_id']
    water_amount = data['water_amount']
    fertilizer_amount = data['fertilizer_amount']
    date_today = datetime.now().strftime('%Y-%m-%d')
    
    df = load_data()
    
    mask = (df['Date'] == date_today) & (df['Plant_ID'] == plant_id)
    if not df[mask].empty:
        df.loc[mask, 'Water_Amount'] = water_amount
        df.loc[mask, 'Fertilizer_Amount'] = fertilizer_amount
    else:
        new_row = {
            'Date': date_today, 
            'Plant_ID': plant_id, 
            'Water_Amount': water_amount, 
            'Fertilizer_Amount': fertilizer_amount
        }
        df = df.append(new_row, ignore_index=True)
    
    save_data(df)
    return jsonify({'status': 'success'})


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)